# General

In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
!pip install openpyxl

In [ ]:
root_path = '/gdrive/My Drive/Colab Data/CRISPR Off Target/'
data_dir = root_path + '2018_DeepCRISPR/'
data_path = data_dir + 'all_off_target.csv'
resource_dir = data_dir + "Resources/"

In [ ]:
import os
import random
import torch
import numpy as np
import copy

seed = 12345

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = 'cpu'
print(device)

cuda


In [ ]:
cd $resource_dir

/gdrive/.shortcut-targets-by-id/1-CPBoDSc88CelqHVwU-GHHjHASJq0kkO/CRISPR Off Target/2018_DeepCRISPR/Resources


#Models

In [ ]:
import torch.nn as nn

class LSTM_Model_Generic(nn.Module):
    def __init__(self, config):
        super(LSTM_Model_Generic,self).__init__()
        # emb_size=256, hidden_size=128, hidden_layers=3, output=2

        self.vocab_size = config["vocab_size"]
        self.emb_size = config["emb_size"]
        self.hidden_size = config["hidden_size"]
        self.lstm_layers = config["lstm_layers"]
        self.bi_lstm = config["bi_lstm"]
        self.reshape = config["reshape"]

        self.number_hidden_layers = config["number_hidder_layers"]
        self.dropout_prob = config["dropout_prob"]
        self.hidden_layers = []

        self.hidden_shape = self.hidden_size*2 if self.bi_lstm else self.hidden_size

        self.embedding = None
        if self.vocab_size > 0:
            self.embedding = nn.Embedding(self.vocab_size, self.emb_size, padding_idx=0)

        self.lstm= nn.LSTM(self.emb_size, self.hidden_size, num_layers=self.lstm_layers,
                            batch_first=True, bidirectional=self.bi_lstm)
#         self.lstm= nn.GRU(self.emb_size, self.hidden_size, num_layers=self.lstm_layers,
#                             batch_first=True, bidirectional=self.bi_lstm)

        start_size = self.hidden_shape

        self.relu = nn.ReLU
        # self.dropout = nn.Dropout(self.dropout_prob)

        for i in range(self.number_hidden_layers):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(start_size, start_size // 2),
                nn.ReLU(),
                nn.Dropout(self.dropout_prob)))

            start_size = start_size // 2

        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        self.output = nn.Linear(start_size,2)


    def forward(self,x):
        dir = 2 if self.bi_lstm else 1
        h = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        c = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)

        if self.embedding is not None:
            x = x.type(torch.LongTensor).to(device)
            x = self.embedding(x)
        elif self.reshape:
            x = x.view(x.shape[0],x.shape[1],1)

        x, (hidden, cell) = self.lstm(x, (h,c))

        x = x[:, -1, :]

        # print(x.shape)
        for i, layer in enumerate(self.hidden_layers):
            x = layer(x)
            # print(x.shape)
        x = self.output(x)
        # print(x.shape)
        return x

In [ ]:
import torch.nn as nn

class RNN_Model_Generic(nn.Module):
    def __init__(self, config, model_type):
        super(RNN_Model_Generic,self).__init__()
        # emb_size=256, hidden_size=128, hidden_layers=3, output=2

        self.model_type = model_type
        self.vocab_size = config["vocab_size"]
        self.emb_size = config["emb_size"]
        self.hidden_size = config["hidden_size"]
        self.lstm_layers = config["lstm_layers"]
        self.bi_lstm = config["bi_lstm"]
        self.reshape = config["reshape"]

        self.number_hidden_layers = config["number_hidder_layers"]
        self.dropout_prob = config["dropout_prob"]
        self.hidden_layers = []

        self.hidden_shape = self.hidden_size*2 if self.bi_lstm else self.hidden_size

        self.embedding = None
        if self.vocab_size > 0:
            self.embedding = nn.Embedding(self.vocab_size, self.emb_size, padding_idx=0)


        if model_type == "LSTM":
            self.lstm = nn.LSTM(self.emb_size, self.hidden_size, num_layers=self.lstm_layers,
                            batch_first=True, bidirectional=self.bi_lstm)
        elif model_type == "GRU":
            self.lstm= nn.GRU(self.emb_size, self.hidden_size, num_layers=self.lstm_layers,
                           batch_first=True, bidirectional=self.bi_lstm)
        else:
            self.lstm= nn.RNN(self.emb_size, self.hidden_size, num_layers=self.lstm_layers,
                           batch_first=True, bidirectional=self.bi_lstm)

        start_size = self.hidden_shape

        self.relu = nn.ReLU
        # self.dropout = nn.Dropout(self.dropout_prob)

        for i in range(self.number_hidden_layers):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(start_size, start_size // 2),
                nn.ReLU(),
                nn.Dropout(self.dropout_prob)))

            start_size = start_size // 2

        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        self.output = nn.Linear(start_size,2)


    def forward(self,x):
        # added for captum's prediction
        softmax = nn.Softmax(dim=1)

        dir = 2 if self.bi_lstm else 1
        h = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        c = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)

        if self.embedding is not None:
            x = x.type(torch.LongTensor).to(device)
            x = self.embedding(x)
        elif self.reshape:
            x = x.view(x.shape[0],x.shape[1],1)

        if self.model_type == "LSTM":
            x, (hidden, cell) = self.lstm(x, (h,c))
        else:
            x, hidden = self.lstm(x, h)

        x = x[:, -1, :]

        # print(x.shape)
        for i, layer in enumerate(self.hidden_layers):
            x = layer(x)
            # print(x.shape)
        x = self.output(x)
        x  = softmax(x)
        # print(x.shape)
        return x

#Training and Evaluation Loop

In [ ]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainerDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs= inputs
        self.targets = torch.from_numpy(targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return torch.Tensor(self.inputs[idx]), self.targets[idx]

In [ ]:
def trainer(config, train_x, train_y, num_epochs=100, batch_size=32, debug=False, lr=0.0001,model_type="LSTM"):
    train_pos_idx = np.where(train_y==1)
    train_neg_idx = np.where(train_y==0)

    train_xp = train_x[train_pos_idx]
    train_xn = train_x[train_neg_idx]

    train_yp = train_y[train_pos_idx]
    train_yn = train_y[train_neg_idx]

    train_dataset_pos = TrainerDataset(train_xp, train_yp)
    train_dataloader_pos = DataLoader(train_dataset_pos, batch_size=batch_size//2, shuffle=True)
    train_dataset_neg = TrainerDataset(train_xn, train_yn)
    train_dataloader_neg = DataLoader(train_dataset_neg, batch_size=batch_size//2, shuffle=True)

    seed = 12345
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    model = RNN_Model_Generic(config, model_type).to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)
    n_total_steps = len(train_dataloader_neg)
    model.train()

    for epoch in range(num_epochs):
        for i, (train_features_neg, train_labels_neg) in enumerate(train_dataloader_neg):
            train_features_pos, train_labels_pos = next(iter(train_dataloader_pos))
            train_features = torch.cat((train_features_pos, train_features_neg),0)
            train_labels = torch.cat((train_labels_pos, train_labels_neg),0)

#             print(train_features.shape, train_labels.shape)

            outputs = model(train_features.to(device))
            loss = criterion(outputs, train_labels.to(device))

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

#             if (i+1) % 2000 == 0 and epoch % 10 == 0:
            if (i+1) % 200 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
                if debug:
                    return model
    return model

In [ ]:
def tester(model, test_x, test_y):
    test_dataset = TrainerDataset(test_x, test_y)
    test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    model.eval()
    results = []
    true_labels = []
    with torch.no_grad():
        for test_features, test_labels in test_dataloader:
            outputs = model(test_features.to(device)).detach().to("cpu")
            results.extend(outputs)
            true_labels.extend(test_labels)
    return true_labels, results

In [ ]:
class Stats:
    def __init__(self):
        self.acc = 0
        self.pre = 0
        self.re = 0
        self.f1 = 0
        self.roc = 0
        self.prc = 0
        self.tn = 0
        self.fp = 0
        self.fn = 0
        self.tp = 0
        self.pred_y = []
    def print(self):
        print('Accuracy: %.4f' %self.acc)
        print('Precision: %.4f' %self.pre)
        print('Recall: %.4f' %self.re)
        print('F1 Score: %.4f' %self.f1)
        print('ROC: %.4f' %self.roc)
        print('PR AUC: %.4f' %self.prc)
        print("Confusion Matrix")
        print(self.tn, "\t", self.fp)
        print(self.fn, "\t", self.tp)

    def formatted_print(self):
        print(self.acc, end="\t")
        print(self.pre, end="\t")
        print(self.re, end="\t")
        print(self.f1, end="\t")
        print(self.roc, end="\t")
        print(self.prc, end="\t")
        print(self.tn, "\t", self.fp, end="\t", sep="")
        print(self.fn, "\t", self.tp, end="\t", sep="")

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

def eval_matrices(model, test_x, test_y, debug = False):
    true_y, results = tester(model, test_x, test_y)
    # predictions = [torch.nn.functional.softmax(r) for r in results]
    predictions = results
    print(predictions[0])
    pred_y = np.array([y[1].item() for y in predictions])
    pred_y_list = []
    test_y = np.array([y.item() for y in true_y])

    for x in pred_y:
        if(x>0.5):
            pred_y_list.append(1)
        else:
            pred_y_list.append(0)

    pred_y_list = np.array(pred_y_list)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_y_list).ravel()
    precision, recall, _ = precision_recall_curve(test_y, pred_y)
    auc_score = auc(recall, precision)
    acc = accuracy_score(test_y, pred_y_list)

    pr = -1
    re = -1
    f1 = -1
    try:
        pr = tp / (tp+fp)
        re = tp / (tp+fn)
        f1 = 2*pr*re / (pr+re)
    except:
        f1 = -1

    stats = Stats()
    stats.acc = acc
    stats.pre = pr
    stats.re = re
    stats.f1 = f1
    stats.roc = roc_auc_score(test_y, pred_y)
    stats.prc = auc_score
    stats.tn = tn
    stats.fp = fp
    stats.fn = fn
    stats.tp = tp

    if debug:
        print('Accuracy: %.4f' %acc)
        print('Precision: %.4f' %pr)
        print('Recall: %.4f' %re)
        print('F1 Score: %.4f' %f1)
        print('ROC:',roc_auc_score(test_y, pred_y))
        print('PR AUC: %.4f' % auc_score)

        print(classification_report(test_y, pred_y_list, digits=4))
        print("Confusion Matrix")
        print(confusion_matrix(test_y, pred_y_list))

    stats.pred_y = pred_y_list

    return stats

# Load Data

In [ ]:
!pip install pickle5

In [ ]:
import pickle5 as pkl
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

in_file = data_dir + "Encoded Data/all_encoded_data_new.pkl"
enc_dict = {}
with open(in_file, "rb") as f:
    enc_dict = pkl.load(f)

data_x = enc_dict['enc_superposed']
data_y = enc_dict['labels']

data_x = np.array(data_x)
data_y = np.array(data_y)

print(data_x.shape)
print(data_y.shape)


train_x, test_x, train_y, test_y = train_test_split(data_x, data_y,
                                                    stratify=data_y,
                                                    test_size=0.20,
                                                    random_state=5)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)


(153233, 23, 4)
(153233,)
(122586, 23, 4) (122586,)
(30647, 23, 4) (30647,)


# Captum Model Interpretation

In [ ]:
!pip install captum

In [ ]:
from captum import attr
torch.backends.cudnn.enabled=False
# device = 'cpu'
# print(stats.pred_y.shape, type(stats.pred_y))
print(test_y.shape, type(test_y))

(30647,) <class 'numpy.ndarray'>


In [ ]:
model_weights = resource_dir + "best_lstm_model.pth"

In [ ]:
best_config = {
    'vocab_size': 0,
    'emb_size': 4,
    'hidden_size': 512,
    'lstm_layers': 1,
    'bi_lstm': True,
    'number_hidder_layers': 2,
    'dropout_prob': 0.4,
    'reshape': False,
    'batch_size': 64,
    'epochs': 50,
    'learning_rate': 0.00010
}

In [ ]:
model = RNN_Model_Generic(best_config, "LSTM").to(device)
model.load_state_dict(torch.load(model_weights))
model.eval()
test_yn = torch.from_numpy(test_y)
interpreter = attr.IntegratedGradients(model)

In [ ]:
stats = eval_matrices(model, test_x, test_y)
print(test_y[0], stats.pred_y[0])

tensor([1.0000e+00, 6.6549e-13])
0 0


In [ ]:
if not os.path.exists(resource_dir + "feature_attribution_dict.pkl"):
    attribution_batch_size = 100
    attributions = []
    for i in range(0,test_x.shape[0], attribution_batch_size):
        start = i
        end = min(i+attribution_batch_size, test_x.shape[0])
        # batch_attributions = interpreter.attribute(torch.Tensor(test_x[start:end]).to(device),
        #                                      target = stats.pred_y[start:end].to(device))
        batch_attributions = interpreter.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device), target = 1)

        # batch_attributions = interpreter.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device), target = 0)
        attributions.extend(batch_attributions.detach().to("cpu").numpy())
        torch.cuda.empty_cache()
        print(start, end)
        # break

    attributions = np.array(attributions)
    print(attributions.shape)

    attribution_dict = {
        "overall_on_1" : attributions,
    }

    with open(resource_dir + "feature_attribution_dict.pkl", "wb") as file:
        pkl.dump(attribution_dict, file)
else:
    torch.cuda.empty_cache()
    with open(resource_dir + "feature_attribution_dict.pkl", "rb") as file:
        attribution_dict = pkl.load(file)
        attributions = attribution_dict["overall_on_1"]

0 100
100 200
200 300
300 400
400 500
500 600
600 700
700 800
800 900
900 1000
1000 1100
1100 1200
1200 1300
1300 1400
1400 1500
1500 1600
1600 1700
1700 1800
1800 1900
1900 2000
2000 2100
2100 2200
2200 2300
2300 2400
2400 2500
2500 2600
2600 2700
2700 2800
2800 2900
2900 3000
3000 3100
3100 3200
3200 3300
3300 3400
3400 3500
3500 3600
3600 3700
3700 3800
3800 3900
3900 4000
4000 4100
4100 4200
4200 4300
4300 4400
4400 4500
4500 4600
4600 4700
4700 4800
4800 4900
4900 5000
5000 5100
5100 5200
5200 5300
5300 5400
5400 5500
5500 5600
5600 5700
5700 5800
5800 5900
5900 6000
6000 6100
6100 6200
6200 6300
6300 6400
6400 6500
6500 6600
6600 6700
6700 6800
6800 6900
6900 7000
7000 7100
7100 7200
7200 7300
7300 7400
7400 7500
7500 7600
7600 7700
7700 7800
7800 7900
7900 8000
8000 8100
8100 8200
8200 8300
8300 8400
8400 8500
8500 8600
8600 8700
8700 8800
8800 8900
8900 9000
9000 9100
9100 9200
9200 9300
9300 9400
9400 9500
9500 9600
9600 9700
9700 9800
9800 9900
9900 10000
10000 10100
10100 10

In [ ]:
feat_attribution = attributions[0]
print(feat_attribution.shape)
feat_attribution = feat_attribution.sum(axis=1)
print(feat_attribution.shape)
print(feat_attribution)
feat_attribution = feat_attribution / np.linalg.norm(feat_attribution)
print(feat_attribution.shape)
print(feat_attribution)

(23, 4)
(23,)
[ 0.06940305  0.03091268  0.06772827  0.02346417  0.00197513 -0.04398288
 -0.06407119 -0.02852685 -0.02722828 -0.00378091 -0.00515271  0.04882731
  0.06230796  0.16130585  0.06240389  0.06459423  0.06643197  0.02774311
  0.20851873  0.04147655 -0.00404112 -0.01631606 -0.00524812]
(23,)
[ 0.20964384  0.09337706  0.20458488  0.07087756  0.00596623 -0.13285784
 -0.19353804 -0.08617024 -0.08224771 -0.01142089 -0.01556466  0.1474913
  0.18821189  0.48725203  0.18850168  0.19511798  0.2006692   0.08380284
  0.62986666  0.12528704 -0.01220691 -0.04928546 -0.01585285]


In [ ]:
feature_importantance_dict = {}
# order=['A','T','C','G']
order_dict = { 0 : 'A', 1:'T', 2:'C', 3:'G', 4:'R' }
print(test_x.shape)

for i in range(test_x.shape[0]):
    ty = int(test_y[i])
    py = int(stats.pred_y[i])

    for j in range(test_x.shape[1]):
        feature_name = 'Pos_' + str(j+1) + '_'
        feat_attribution = attributions[i]
        feat_attribution = feat_attribution.sum(axis=1)
        feat_attribution = feat_attribution / np.linalg.norm(feat_attribution)

        for k in range(test_x.shape[2]):
            if test_x[i][j][k] > 0:
                feature_name += order_dict[k]

        if feature_name not in feature_importantance_dict:
            feature_importantance_dict[feature_name] = {'tpc':0, 'tnc':0, 'fpc':0, 'fnc':0,
                                                        'tps':0, 'tns':0, 'fps':0, 'fns':0,}

        if ty == 0 and py == 0:
            feature_importantance_dict[feature_name]['tnc'] += 1
            feature_importantance_dict[feature_name]['tns'] += feat_attribution[j]

        elif ty == 0 and py == 1:
            feature_importantance_dict[feature_name]['fpc'] += 1
            feature_importantance_dict[feature_name]['fps'] += feat_attribution[j]

        elif ty == 1 and py == 0:
            feature_importantance_dict[feature_name]['fnc'] += 1
            feature_importantance_dict[feature_name]['fns'] += feat_attribution[j]

        elif ty == 1 and py == 1:
            feature_importantance_dict[feature_name]['tpc'] += 1
            feature_importantance_dict[feature_name]['tps'] += feat_attribution[j]

(30647, 23, 4)


In [ ]:
features = list(feature_importantance_dict.keys())
tp = []
tps = []
tn = []
tns = []
fp = []
fps = []
fn = []
fns = []
print(len(features))

218


In [ ]:
for f in features:
    tn.append(feature_importantance_dict[f]['tnc'])
    tns.append(feature_importantance_dict[f]['tns'])

    fp.append(feature_importantance_dict[f]['fpc'])
    fps.append(feature_importantance_dict[f]['fps'])

    fn.append(feature_importantance_dict[f]['fnc'])
    fns.append(feature_importantance_dict[f]['fns'])

    tp.append(feature_importantance_dict[f]['tpc'])
    tps.append(feature_importantance_dict[f]['tps'])

In [ ]:
import pandas as pd
reformed_dict = {
    "features" : features,
    "tnc" : tn,
    "tns" : tns,
    "fpc" : fp,
    "fps" : fps,
    "fnc" : fn,
    "fns" : fns,
    "tpc" : tp,
    "tps" : tps
}

df = pd.DataFrame(reformed_dict)
print(df.shape)
print(df.iloc[0])

(218, 9)
features      Pos_1_CG
tnc               1953
tns         182.117507
fpc                  4
fps          -0.106583
fnc                  7
fns           1.859818
tpc                  8
tps           0.059663
Name: 0, dtype: object


In [ ]:
df.to_excel(resource_dir + 'feature_importance_neg.xlsx')

# Layer Attribution

In [ ]:
print(model)

RNN_Model_Generic(
  (lstm): LSTM(4, 512, batch_first=True, bidirectional=True)
  (hidden_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.4, inplace=False)
    )
    (1): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.4, inplace=False)
    )
  )
  (output): Linear(in_features=256, out_features=2, bias=True)
)


In [ ]:
print(model.hidden_layers[0][1])
print(model.hidden_layers[1][1])

ReLU()
ReLU()


In [ ]:
torch.cuda.empty_cache()

In [ ]:
interpreter_hl1 = attr.LayerConductance(model, model.hidden_layers[0][1])
interpreter_hl2 = attr.LayerConductance(model, model.hidden_layers[1][1])
attribution_batch_size = 25
attributions_hl1 = []
attributions_hl2 = []

torch.cuda.empty_cache()
if not os.path.exists(resource_dir + "layer_attribution_dict.pkl"):
    for i in range(0,test_x.shape[0], attribution_batch_size):
        start = i
        end = min(i+attribution_batch_size, test_x.shape[0])

        batch_attributions_hl1 = interpreter_hl1.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device),
                                            target = 1)
        attributions_hl1.extend(batch_attributions_hl1.detach().to("cpu").numpy())
        torch.cuda.empty_cache()

        batch_attributions_hl2 = interpreter_hl2.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device),
                                            target = 1)
        attributions_hl2.extend(batch_attributions_hl2.detach().to("cpu").numpy())
        torch.cuda.empty_cache()

        print(start, end)
#         break

    attributions_hl1 = np.array(attributions_hl1)
    print(attributions_hl1.shape)

    attributions_hl2 = np.array(attributions_hl2)
    print(attributions_hl2.shape)

    layer_attributions = {
        "hl1" : attributions_hl1,
        "hl2" : attributions_hl2
    }

    all_attributions = np.hstack((attributions_hl1, attributions_hl2))
    print(all_attributions.shape)

    with open(resource_dir + "layer_attribution_dict.pkl", "wb") as file:
        pkl.dump(layer_attributions, file)

else:
    with open(resource_dir + "layer_attribution_dict.pkl", "rb") as file:
        layer_attributions = pkl.load(file)
        attributions_hl1 = layer_attributions["hl1"]
        print(attributions_hl1.shape)

        attributions_hl2 = layer_attributions["hl2"]
        print(attributions_hl2.shape)

        all_attributions = np.hstack((attributions_hl1, attributions_hl2))
        print(all_attributions.shape)


0 25
25 50
50 75
75 100
100 125
125 150
150 175
175 200
200 225
225 250
250 275
275 300
300 325
325 350
350 375
375 400
400 425
425 450
450 475
475 500
500 525
525 550
550 575
575 600
600 625
625 650
650 675
675 700
700 725
725 750
750 775
775 800
800 825
825 850
850 875
875 900
900 925
925 950
950 975
975 1000
1000 1025
1025 1050
1050 1075
1075 1100
1100 1125
1125 1150
1150 1175
1175 1200
1200 1225
1225 1250
1250 1275
1275 1300
1300 1325
1325 1350
1350 1375
1375 1400
1400 1425
1425 1450
1450 1475
1475 1500
1500 1525
1525 1550
1550 1575
1575 1600
1600 1625
1625 1650
1650 1675
1675 1700
1700 1725
1725 1750
1750 1775
1775 1800
1800 1825
1825 1850
1850 1875
1875 1900
1900 1925
1925 1950
1950 1975
1975 2000
2000 2025
2025 2050
2050 2075
2075 2100
2100 2125
2125 2150
2150 2175
2175 2200
2200 2225
2225 2250
2250 2275
2275 2300
2300 2325
2325 2350
2350 2375
2375 2400
2400 2425
2425 2450
2450 2475
2475 2500
2500 2525
2525 2550
2550 2575
2575 2600
2600 2625
2625 2650
2650 2675
2675 2700
2700 27

In [ ]:
layer_importance_dict = []

for j in range(all_attributions.shape[1]):
        layer = 1 if j<attributions_hl1.shape[1] else 2
        neuron = j if j<attributions_hl1.shape[1] else j - attributions_hl1.shape[1]

        neuron_dict = {
            "Layer": layer,
            "Neuron" : neuron,
            "Neuron_Name": "HL_" + str(layer) + "_N_" + str(neuron),
            "pos_score": 0,
            "pos_count": 0,
            "neg_score": 0,
            "neg_count": 0,
            "total_score": 0,
            "total_count": 0
        }

        layer_importance_dict.append(neuron_dict)

print(len(layer_importance_dict))
print(layer_importance_dict[0]["Neuron_Name"])
print(layer_importance_dict[511]["Neuron_Name"])
print(layer_importance_dict[512]["Neuron_Name"])
print(layer_importance_dict[-1]["Neuron_Name"])

768
HL_1_N_0
HL_1_N_511
HL_2_N_0
HL_2_N_255


In [ ]:
for i in range(test_x.shape[0]):
    ty = int(test_y[i])
    py = int(stats.pred_y[i])

    for j in range(len(layer_importance_dict)):
        layer_importance_dict[j]["total_score"] += all_attributions[i][j]
        layer_importance_dict[j]["total_count"] += 1

        if py > 0:
            layer_importance_dict[j]["pos_score"] += all_attributions[i][j]
            layer_importance_dict[j]["pos_count"] += 1
        else:
            layer_importance_dict[j]["neg_score"] += all_attributions[i][j]
            layer_importance_dict[j]["neg_count"] += 1

In [ ]:
import pandas as pd

df = pd.DataFrame.from_dict(layer_importance_dict)
print(df.shape)
print(df.iloc[0])
df.to_excel(resource_dir + "Layer_Importance.xlsx")

(768, 9)
Layer                  1
Neuron                 0
Neuron_Name     HL_1_N_0
pos_score      -0.037365
pos_count            109
neg_score     -83.104712
neg_count          30538
total_score   -83.142076
total_count        30647
Name: 0, dtype: object


# Neuron Imoportance

In [ ]:
interpreter_n1 = attr.NeuronConductance(model, model.hidden_layers[0][1])
interpreter_n2 = attr.NeuronConductance(model, model.hidden_layers[0][1])

attribution_batch_size = 25
attributions_n1 = []
attributions_n2 = []

pos_neuron = 310
neg_neuron = 215

if not os.path.exists(resource_dir + "neuron_attribution_dict.pkl"):
    torch.cuda.empty_cache()
    for i in range(0,test_x.shape[0], attribution_batch_size):
        start = i
        end = min(i+attribution_batch_size, test_x.shape[0])

        batch_attributions_n1 = interpreter_n1.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device),
                                                neuron_selector = pos_neuron,
                                                target = 1)
        attributions_n1.extend(batch_attributions_n1.detach().to("cpu").numpy())
        torch.cuda.empty_cache()

        batch_attributions_n2 = interpreter_n2.attribute(torch.Tensor(test_x[start:end]).requires_grad_().to(device),
                                                neuron_selector = neg_neuron,
                                                target = 1)
        attributions_n2.extend(batch_attributions_n2.detach().to("cpu").numpy())
        torch.cuda.empty_cache()

        print(start, end)

    attributions_n1 = np.array(attributions_n1)
    print(attributions_n1.shape)

    attributions_n2 = np.array(attributions_n2)
    print(attributions_n2.shape)

    neuron_attributions = {
        "n1" : attributions_n1,
        "n2" : attributions_n2
    }
    #         break
    with open(resource_dir + "neuron_attribution_dict.pkl", "wb") as file:
        pkl.dump(layer_attributions, file)

else:
    with open(resource_dir + "neuron_attribution_dict.pkl", "rb") as file:
        neuron_attributions = pkl.load(file)
        attributions_n1 = neuron_attributions["n1"]
        attributions_n1 = neuron_attributions["n2"]
        print(attributions_n1.shape)
        print(attributions_n2.shape)

0 25
25 50
50 75
75 100
100 125
125 150
150 175
175 200
200 225
225 250
250 275
275 300
300 325
325 350
350 375
375 400
400 425
425 450
450 475
475 500
500 525
525 550
550 575
575 600
600 625
625 650
650 675
675 700
700 725
725 750
750 775
775 800
800 825
825 850
850 875
875 900
900 925
925 950
950 975
975 1000
1000 1025
1025 1050
1050 1075
1075 1100
1100 1125
1125 1150
1150 1175
1175 1200
1200 1225
1225 1250
1250 1275
1275 1300
1300 1325
1325 1350
1350 1375
1375 1400
1400 1425
1425 1450
1450 1475
1475 1500
1500 1525
1525 1550
1550 1575
1575 1600
1600 1625
1625 1650
1650 1675
1675 1700
1700 1725
1725 1750
1750 1775
1775 1800
1800 1825
1825 1850
1850 1875
1875 1900
1900 1925
1925 1950
1950 1975
1975 2000
2000 2025
2025 2050
2050 2075
2075 2100
2100 2125
2125 2150
2150 2175
2175 2200
2200 2225
2225 2250
2250 2275
2275 2300
2300 2325
2325 2350
2350 2375
2375 2400
2400 2425
2425 2450
2450 2475
2475 2500
2500 2525
2525 2550
2550 2575
2575 2600
2600 2625
2625 2650
2650 2675
2675 2700
2700 27

In [ ]:
import math
neuron_importantance_dict = {}
order_dict = { 0 : 'A', 1:'T', 2:'C', 3:'G', 4:'R' }
print(test_x.shape)

for i in range(test_x.shape[0]):
    ty = int(test_y[i])
    py = int(stats.pred_y[i])

    n1_attribution = attributions_n1[i]
    n1_attribution = n1_attribution.sum(axis=1)
    n1_attribution = n1_attribution / np.linalg.norm(n1_attribution)

    n2_attribution = attributions_n2[i]
    n2_attribution = n2_attribution.sum(axis=1)
    n2_attribution = n2_attribution / np.linalg.norm(n2_attribution)



    for j in range(test_x.shape[1]):
        feature_name = 'Pos_' + str(j+1) + '_'

        for k in range(test_x.shape[2]):
            if test_x[i][j][k] > 0:
                feature_name += order_dict[k]

        if feature_name not in neuron_importantance_dict:
            neuron_importantance_dict[feature_name] = {'n1_tot_c': 0, 'n1_tot_s': 0,
                                                       'n2_tot_c': 0, 'n2_tot_s': 0}

        # if py < 0.5:
        neuron_importantance_dict[feature_name]['n2_tot_c'] += 1
        neuron_importantance_dict[feature_name]['n2_tot_s'] += 0 if math.isnan(n2_attribution[j]) else n2_attribution[j]
        # else:
        neuron_importantance_dict[feature_name]['n1_tot_c'] += 1
        neuron_importantance_dict[feature_name]['n1_tot_s'] += 0 if math.isnan(n1_attribution[j]) else n1_attribution[j]

(30647, 23, 4)


In [ ]:
features = list(neuron_importantance_dict.keys())
n1_tot_c = []
n1_tot_s = []

n2_tot_c = []
n2_tot_s = []


print(len(features))

for f in features:
    n1_tot_c.append(neuron_importantance_dict[f]['n1_tot_c'])
    n1_tot_s.append(neuron_importantance_dict[f]['n1_tot_s'])

    n2_tot_c.append(neuron_importantance_dict[f]['n2_tot_c'])
    n2_tot_s.append(neuron_importantance_dict[f]['n2_tot_s'])


reformed_dict = {
    "features" : features,
    "n1_tot_c" : n1_tot_c,
    "n1_tot_s" : n1_tot_s,
    "n2_tot_c" : n2_tot_c,
    "n2_tot_s" : n2_tot_s
}

df = pd.DataFrame(reformed_dict)
print(df.shape)
print(df.iloc[0])

218
(218, 5)
features      Pos_1_CG
n1_tot_c          1972
n1_tot_s   -153.062445
n2_tot_c          1972
n2_tot_s    -170.42968
Name: 0, dtype: object


In [ ]:
df.to_excel(resource_dir + "Neuron_Importance.xlsx")